# Titanic

In [ ]:
import pandas as pd
import re
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load in the train and test datasets
train = pd.read_csv('../input/titanic-machine-learning-from-disaster/test.csv')
test = pd.read_csv('../input/titanic-machine-learning-from-disaster/test.csv')

# Store our passenger ID for easy access
PassengerId = test['PassengerId']

train.head(3)

In [ ]:
full_data = [train, test]

# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

In [ ]:
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
g = sns.pairplot(train[[u'Survived', u'Pclass', u'Sex', u'Age', u'Parch', u'Fare', 
                        u'Embarked', u'FamilySize', u'Title']], 
                 hue='Survived', palette = 'seismic', size=1.2, plot_kws=dict(s=10))
g.set(xticklabels=[])

Итак, у нас получились два датасета с новыми признаками. Теперь приступим к построению модели.

### Построение модели

### 1.

Воспользуйтесь вашим алгоритмом стекинга из предыдущего домашнего задания. В качестве базовых алгоритмов используйте RandomForestClassifier, SVC, GradientBoostingClassifier и LogisticRegression; в качестве мета-алгоритма - XGBoost.

Разделите данные train на тренировочную и валидационную выборки с random_state=17 и параметром разбиения test_size=.3 (в качестве целевой переменной возьмите столбец Survived, а в качестве признаков - все остальные столбцы).

Ниже приведены параметры для каждого из базовых алгоритмов, которые необходимо настроить на 5-кратной кросс-валидации с помощью GridSearchCV:

In [ ]:
from sklearn.model_selection import (GridSearchCV,
                                     train_test_split,
                                     StratifiedKFold)

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

# параметры базовых алгоритмов
gbc_params = {'learning_rate': np.arange(0.1, 0.6, 0.1)} # GradientBoostingClassifier

rfc_params = {'n_estimators': range(10, 100, 10), # RandomForestClassifier
              'min_samples_leaf': range(1, 5)}

svc_params = {'kernel': ['linear', 'rbf'], # SVC
              'C': np.arange(0.1, 1, 0.2)}

lr_params = {'C': np.arange(0.5, 1, 0.1)}

skf = StratifiedKFold(n_splits=5, random_state=17)

In [ ]:
# получение тренировочных и валидационных данных
x_train, x_valid, y_train, y_valid = train_test_split(train.iloc[:,1:].values,
                                                      train.iloc[:,0].values,
                                                      random_state=17,
                                                      test_size=.3)

### 2.
1. Определите объект GridSearchCV для всех приведенных параметров каждого алгоритма (в гиперпараметрах алгоритма при его определении, если возможно, укажите random_state=17). Параметр cv устанавливайте равным skf.

2. Обучите каждый из объектов из 1-го пункта на получившейся при разбиении тренировочной выборке. Выведите лучшее сочетание параметров для каждого из алгоритмов.

3. Для каждого обученного алгоритма получите предсказания на валидационных данных и выведите метрику качества, которая соответствует метрике оценки соревнования.

In [ ]:
from sklearn.metrics import accuracy_score

# инициализация моделей c GridSearchCV
gbc = GradientBoostingClassifier(random_state=17)
gbc_gs = GridSearchCV(gbc, gbc_params, cv=skf)
rfc = RandomForestClassifier(random_state=17)
rfc_gs = GridSearchCV(rfc, rfc_params, cv=skf)
svc = SVC(random_state=17)
svc_gs = GridSearchCV(svc, svc_params, cv=skf)
lr = LogisticRegression(random_state=17)
lr_gs = GridSearchCV(lr, lr_params, cv=skf)

meta = XGBClassifier(n_estimators=40)

models = [gbc_gs, rfc_gs, svc_gs, lr_gs]

for model in models:
    model.fit(x_train, y_train)
    print('Model=', model)
    print(model.best_params_)
    print(f'accuracy={accuracy_score(y_valid, model.predict(x_valid))}')
    print('---------------------')

### 3.
С помощью GridSearchCV и указанных ниже параметров настройте мета-алгоритм на мета-признаках (используйте 5-кратную валидацию и random_state=17 при определении алгоритма). Матрицу метапризнаков получите из предсказаний, полученных в предыдущем пункте на валидационных данных базовыми алгоритмами. Выведите лучшие параметры.

In [ ]:
xgb_params = {'n_estimators': range(10, 100, 5),
              'eta': np.arange(0.1, 1., .1),
              'min_child_weight': range(1, 10, 1),
              'subsample': np.arange(0.1, 1., 0.2)}

xgb = XGBClassifier(random_state=17, eval_metric='error')
xbg_gs = GridSearchCV(xgb, xgb_params, cv=5)

meta_mtrx = np.empty((x_valid.shape[0], len(models)))
for n, model in enumerate(models):
    meta_mtrx[:, n] = model.predict(x_valid)


In [ ]:
xbg_gs.fit(meta_mtrx, y_valid)

In [ ]:
xbg_gs.best_params_

### 4.
На основе алгоритма из предыдущего домашнего задания постройте стекинг (используйте 5-кратную кросс-валидацию) для всех моделей с наилучшими подобранными параметрами. В качестве тренировочных данных используйте весь датасет train.csv, а в качестве тестовых - весь датасет test.csv. Сделайте прогноз мета-алгоритма для test.csv.

In [ ]:
from sklearn.model_selection import cross_val_predict


def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, cv=5):
    """Функция стекинга
    Аргументы:
    models -- список предварительно инициализированных моделей, которые будут использоваться
    meta_alg -- метаалгоритм стекинга
    data_train -- тренировочные признаки
    targets_train -- целевые переменные тренировочных данных
    data_test -- тестовые признаки
    targets_test  -- целевые переменные тестовых данных (default None)
    random_state -- воспроизводимость модели (default None)
    test_size -- размер тестовой выборки тренировочных данных (default None)
    cv -- коэффициент кросс-валидации (default 5)
    metrics -- список требуемых метрик точности модели при наличии целевых переменнныъ тестовых данных (default [roc_auc_score])
    """
    meta_mtrx = np.empty((targets_train.shape[0], len(models)))
#   заполнение матрицы метапризнаков предсказаниями через кроссвалидацию cross_val_predict
    for n, model in enumerate(models):
#  заполнение матрицы метапризнаков данными кроссвалидации и обучение моделей на полном тестовом датасете
        meta_mtrx[:, n] = cross_val_predict(model,
                                            data_train,
                                            targets_train,
                                            cv=cv,
                                            method='predict')
        model.fit(data_train, targets_train)

#  обучение модели на матрице метапризнаков и целевых значениях валидационных данных
    meta_alg.fit(meta_mtrx, targets_train)
#  предсказания базовых моделей на тестовых данных
    meta_mtrx_test = np.empty((data_test.shape[0], len(models))) 
    for n, model in enumerate(models):
        meta_mtrx_test[:, n] = model.predict(data_test)
#  предсказания метамодели на meta_mtrx_test
    meta_predict = meta_alg.predict(meta_mtrx_test)
#  если известны значения тестовых данных, вернуть предсказания и значения всех заданных метрик, иначе вернуть только предсказания
    return meta_predict

In [ ]:
models = [gbc, rfc, svc, lr]

predictions = stacking(models,
                       xgb,
                       train.iloc[:, 1:].values,
                       train.iloc[:,0].values,
                       test.iloc[:, 1:].values)

### 5.
С помощью нижеприведенной функции сформируйте файл посылки для соревнования и отправьте на Kaggle.

In [ ]:
def write_to_submission_file(predictions, PassengerID, out_file='Submission.csv', columns=['PassengerID', 'Survived']):
    predicted_df = pd.DataFrame(np.array([PassengerId, predictions]).T, columns=columns)
    predicted_df.to_csv(out_file, index=False)

In [ ]:
write_to_submission_file(predictions, PassengerId)

### 6.
Каков результат score, полученного на соревновании?

Ваш ответ: